In [ ]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import GaussianNB
from ipynb.fs.defs.MMRWPC import MMRWPC
from ipynb.fs.defs.MMPC import MMPC
from ipynb.fs.defs.MMPC_opt import MMPC_opt
import time
import os
from ipynb.fs.defs.GraphReader import GraphReader

In [ ]:
class MMPC_sequential_testing():
    
    def __init__(self, mmpc_models, model_names, testing_dataset_route, log_route, selected_nodes_route,
                 graph_info_route, n_shuffles):
        
        self._mmpc_models = mmpc_models
        self._model_names = model_names
        self._testing_dataset_route = testing_dataset_route
        self._log_route = log_route
        self._n_shuffles = n_shuffles
        self._graph_reader = GraphReader(graph_info_route)
        self._files_dict = None
        self._results = None
        preselected_nodes = []
        try:
            ff = open(selected_nodes_route,'r')
            for e in ff.read().split('\n'):
                spl_line = e.split(';')
                if (len(spl_line[0])==0):
                    continue
                preselected_nodes.append(spl_line[0])
            ff.close()
        except:
            print("Error opening preselected nodes file")
            return
        
        self._selected_nodes = preselected_nodes
        self.__init_files()
        print("MMPC_Testing object succesfully created.")
        
        return
    
    def __init_files(self):
        
        files_dict = {}
        for filename in os.listdir(self._testing_dataset_route):
            temp_df = pd.read_csv(
                self._testing_dataset_route+
                filename,delimiter='  ',header=None, engine='python')
            graph_heads = []
            for k in range(0,len(temp_df.columns)):
                graph_heads.append('Node'+str(k+1))
            temp_df.columns = graph_heads
            files_dict[filename] = temp_df
            print(filename)
        
        self._files_dict = files_dict
        return
    
    def getResults(self):
        if self._results is None:
            print("No calculated results yet.")
            
            return []
        
        return list(self._results)
    
    def __write_out_pobdict(self, pobdict):
        ff = open(self._log_route+'sequential_results_'+pobdict['dataset'].replace('.txt','').replace('_','')+'_'+pobdict['TargetNode']+'.txt', 'w')
        model_results = pobdict['models_results']
        for e in model_results:
            ff.write(e['model_name']+';')
            ff.write(str(e['model_time'])+';')
            ff.write(str(e['pc_distance'])+';')
            for node in e['model_CPC']:
                ff.write(node['name']+',')
            ff.write('\n')
        ff.close()
        
        return
    
    def __evaluate(self, PCs):
        
        processed_pc_list = []
        for file_dict in self._files_dict:
            nodes_to_process = [x for x in PCs if x['dataset'] == file_dict]
            pcs_index = [int(x['TargetNode'].split('@')[0].split('Node')[1])-1 for x in nodes_to_process]
            for model_c in range(0,len(self._mmpc_models)):
                self._mmpc_models[model_c].fit(self._files_dict[file_dict])
            
            
            for nodes_c in range(0, len(nodes_to_process)):
                pob_dict = {}
                pob_dict['dataset'] = file_dict
                pob_dict['TargetNode'] = nodes_to_process[nodes_c]['TargetNode']
                pob_dict['models_results'] = []
                for model_c in range(0,len(self._mmpc_models)):
                    for shuffle_c in range(0,self._n_shuffles):
                        self._mmpc_models[model_c].shuffle_rows()
                        model_results = self._mmpc_models[model_c].evaluate(pcs_index[nodes_c])
                        results_dict = {}
                        results_dict['model_name'] = self._model_names[model_c]
                        results_dict['model_time'] = model_results['time']
                        results_dict['model_CPC'] = model_results['CPC']
                        results_dict['pc_distance'] = self._graph_reader.getDistance(pcs_index[nodes_c],[x['name'].split('@')[0] for x in results_dict['model_CPC']])
                        pob_dict['models_results'].append(results_dict)
                
                # write out results in formatted file
                self.__write_out_pobdict(pob_dict)
                
                # save info
                processed_pc_list.append(pob_dict)        
            
                
        return processed_pc_list
    
    def __getNodeDicts(self):
        """
        Create list of parameters to pass
        for a parallel execution including
        the file to examine and the node to examine
        """
        pob_dicts = []
        for node in self._selected_nodes:
            for filename in os.listdir(self._testing_dataset_route):
                pob_dict = {}
                pob_dict['dataset'] = filename
                pob_dict['TargetNode'] = node
                pob_dicts.append(pob_dict)
        
        return pob_dicts
    
    def test_models(self):
        PCs = self.__getNodeDicts()
        model_pcs = self.__evaluate(PCs)
#         scored_pcs = Parallel(n_jobs=10)(map(delayed(self.__scorePC),PCs))
#         self._results = scored_pcs
        self._results = model_pcs
        
        return
    

In [ ]:
seq_testing  = MMPC_sequential_testing(
                            [MMRWPC(0.05,0.0,1), MMPC_opt(0.05),MMPC(0.05)],
                            ['MMRWPC','MMPCOPT','MMPC'],
                            '/home/a20114261/sdelrio/alarm_datasets/Alarm10/', 
                            '/home/a20114261/sdelrio/alarm_datasets/log_sequential_results/',
                            '/home/a20114261/sdelrio/alarm_datasets/log_results/mmpc_partitioned/alarm10_binary_nodes_selected.txt',
                            '/home/a20114261/sdelrio/Alarm10_graph.txt',
                            5)

In [ ]:
seq_testing.test_models()

CPC size after filtering: 5
['Node348@3', 'Node163@3', 'Node21@4', 'Node319@4', 'Node153@3']

Universe actual size:
1
CPC size before filtering: 6
Size set for community selection phase will be of maximum: 4
['Node348@3']
n_set of length: 1
['Node163@3', 'Node153@3']
n_set of length: 2
['Node319@4', 'Node21@4', 'Node322@3']
n_set of length: 3
CPC size after filtering: 6
['Node348@3', 'Node153@3', 'Node163@3', 'Node322@3', 'Node21@4', 'Node319@4']

Universe actual size:
0
CPC size before filtering: 7
Size set for community selection phase will be of maximum: 4
['Node5@3', 'Node21@4']
n_set of length: 2
['Node348@3']
n_set of length: 1
['Node319@4', 'Node322@3']
n_set of length: 2
['Node163@3', 'Node153@3']
n_set of length: 2
CPC size after filtering: 7
['Node21@4', 'Node5@3', 'Node348@3', 'Node322@3', 'Node319@4', 'Node153@3', 'Node163@3']
Entering Phase I
MMPC_beggining: 
369

Universe actual size:
7
CPC size before filtering: 1
CPC size after filtering: 1
['Node153@3']

Universe actua


Universe actual size:
1
CPC actual size:
5
CPC contents:
['Node153@3', 'Node163@3', 'Node5@3', 'Node322@3', 'Node348@3']

Universe actual size:
0
CPC actual size:
6
CPC contents:
['Node153@3', 'Node163@3', 'Node5@3', 'Node322@3', 'Node348@3', 'Node319@4']

Entering Phase II
Entering Phase I
MMPC_beggining: 
369

Universe actual size:
7
CPC actual size:
1
CPC contents:
['Node153@3']

Universe actual size:
5
CPC actual size:
2
CPC contents:
['Node153@3', 'Node163@3']

Universe actual size:
4
CPC actual size:
3
CPC contents:
['Node153@3', 'Node163@3', 'Node5@3']

Universe actual size:
2
CPC actual size:
4
CPC contents:
['Node153@3', 'Node163@3', 'Node5@3', 'Node322@3']

Universe actual size:
1
CPC actual size:
5
CPC contents:
['Node153@3', 'Node163@3', 'Node5@3', 'Node322@3', 'Node348@3']

Universe actual size:
0
CPC actual size:
6
CPC contents:
['Node153@3', 'Node163@3', 'Node5@3', 'Node322@3', 'Node348@3', 'Node319@4']

Entering Phase II
Entering Phase I
MMPC_beggining: 
369

Universe 


Universe actual size:
3
CPC size before filtering: 8
Size set for community selection phase will be of maximum: 5
['Node113@4']
n_set of length: 1
['Node126@3']
n_set of length: 1
['Node5@3', 'Node81@3']
n_set of length: 2
['Node215@2']
n_set of length: 1
['Node292@3', 'Node156@3']
n_set of length: 2
['Node155@3']
n_set of length: 1
CPC size after filtering: 8
['Node113@4', 'Node126@3', 'Node81@3', 'Node5@3', 'Node215@2', 'Node156@3', 'Node292@3', 'Node155@3']

Universe actual size:
1
CPC size before filtering: 9
Size set for community selection phase will be of maximum: 5
['Node81@3', 'Node5@3']
n_set of length: 2
['Node215@2']
n_set of length: 1
['Node126@3']
n_set of length: 1
['Node156@3', 'Node292@3', 'Node155@3']
n_set of length: 3
['Node113@4']
n_set of length: 1
['Node352@3']
n_set of length: 1
CPC size after filtering: 9
['Node5@3', 'Node81@3', 'Node215@2', 'Node126@3', 'Node155@3', 'Node292@3', 'Node156@3', 'Node113@4', 'Node352@3']

Universe actual size:
0
CPC size before f

Entering Phase I
MMPC_beggining: 
369

Universe actual size:
12
CPC size before filtering: 1
CPC size after filtering: 1
['Node155@3']

Universe actual size:
10
CPC size before filtering: 2
Size set for community selection phase will be of maximum: 2
['Node155@3', 'Node156@3']
n_set of length: 2
CPC size after filtering: 2
['Node156@3', 'Node155@3']

Universe actual size:
9
CPC size before filtering: 3
Size set for community selection phase will be of maximum: 3
['Node155@3', 'Node156@3', 'Node5@3']
n_set of length: 3
CPC size after filtering: 3
['Node5@3', 'Node156@3', 'Node155@3']

Universe actual size:
7
CPC size before filtering: 4
Size set for community selection phase will be of maximum: 3
['Node155@3', 'Node156@3', 'Node215@2', 'Node5@3']
n_set of length: 4
CPC size after filtering: 4
['Node5@3', 'Node215@2', 'Node156@3', 'Node155@3']

Universe actual size:
6
CPC size before filtering: 5
Size set for community selection phase will be of maximum: 4
['Node155@3', 'Node156@3', 'Nod

Entering Phase I
MMPC_beggining: 
369

Universe actual size:
12
CPC actual size:
1
CPC contents:
['Node155@3']

Universe actual size:
10
CPC actual size:
2
CPC contents:
['Node155@3', 'Node156@3']

Universe actual size:
8
CPC actual size:
3
CPC contents:
['Node155@3', 'Node156@3', 'Node196@2']

Universe actual size:
6
CPC actual size:
4
CPC contents:
['Node155@3', 'Node156@3', 'Node196@2', 'Node5@3']

Universe actual size:
5
CPC actual size:
5
CPC contents:
['Node155@3', 'Node156@3', 'Node196@2', 'Node5@3', 'Node215@2']

Universe actual size:
4
CPC actual size:
6
CPC contents:
['Node155@3', 'Node156@3', 'Node196@2', 'Node5@3', 'Node215@2', 'Node81@3']

Universe actual size:
1
CPC actual size:
7
CPC contents:
['Node155@3', 'Node156@3', 'Node196@2', 'Node5@3', 'Node215@2', 'Node81@3', 'Node292@3']

Universe actual size:
0
CPC actual size:
8
CPC contents:
['Node155@3', 'Node156@3', 'Node196@2', 'Node5@3', 'Node215@2', 'Node81@3', 'Node292@3', 'Node352@3']

Entering Phase II
Entering Phase

...

Entering Phase I
MMPC_beggining: 
369

Universe actual size:
9
CPC actual size:
1
CPC contents:
['Node14@2']

Universe actual size:
7
CPC actual size:
2
CPC contents:
['Node14@2', 'Node63@3']

Universe actual size:
6
CPC actual size:
3
CPC contents:
['Node14@2', 'Node63@3', 'Node13@3']

Universe actual size:
5
CPC actual size:
4
CPC contents:
['Node14@2', 'Node63@3', 'Node13@3', 'Node358@4']

Universe actual size:
4
CPC actual size:
5
CPC contents:
['Node14@2', 'Node63@3', 'Node13@3', 'Node358@4', 'Node126@3']

Universe actual size:
3
CPC actual size:
6
CPC contents:
['Node14@2', 'Node63@3', 'Node13@3', 'Node358@4', 'Node126@3', 'Node269@3']

Universe actual size:
2
CPC actual size:
7
CPC contents:
['Node14@2', 'Node63@3', 'Node13@3', 'Node358@4', 'Node126@3', 'Node269@3', 'Node334@4']

Universe actual size:
1
CPC actual size:
8
CPC contents:
['Node14@2', 'Node63@3', 'Node13@3', 'Node358@4', 'Node126@3', 'Node269@3', 'Node334@4', 'Node330@3']

Universe actual size:
0
CPC actual size:
9


Universe actual size:
2
CPC actual size:
1
CPC contents:
['Node30@2']

Universe actual size:
1
CPC actual size:
2
CPC contents:
['Node30@2', 'Node226@4']

Universe actual size:
0
CPC actual size:
3
CPC contents:
['Node30@2', 'Node226@4', 'Node52@3']

Entering Phase II
Entering Phase I
MMPC_beggining: 
369

Universe actual size:
2
CPC actual size:
1
CPC contents:
['Node30@2']

Universe actual size:
1
CPC actual size:
2
CPC contents:
['Node30@2', 'Node226@4']

Universe actual size:
0
CPC actual size:
3
CPC contents:
['Node30@2', 'Node226@4', 'Node52@3']

Entering Phase II
Entering Phase I
MMPC_beggining: 
369

Universe actual size:
2
CPC actual size:
1
CPC contents:
['Node30@2']

Universe actual size:
1
CPC actual size:
2
CPC contents:
['Node30@2', 'Node226@4']

Universe actual size:
0
CPC actual size:
3
CPC contents:
['Node30@2', 'Node226@4', 'Node52@3']

Entering Phase II
Entering Phase I
MMPC_beggining: 
369

Universe actual size:
2
CPC actual size:
1
CPC contents:
['Node30@2']

Univ

Entering Phase I
MMPC_beggining: 
369

Universe actual size:
9
CPC actual size:
1
CPC contents:
['Node298@4']

Universe actual size:
7
CPC actual size:
2
CPC contents:
['Node298@4', 'Node317@4']

Universe actual size:
3
CPC actual size:
3
CPC contents:
['Node298@4', 'Node317@4', 'Node304@3']

Universe actual size:
1
CPC actual size:
4
CPC contents:
['Node298@4', 'Node317@4', 'Node304@3', 'Node34@3']

Universe actual size:
0
CPC actual size:
5
CPC contents:
['Node298@4', 'Node317@4', 'Node304@3', 'Node34@3', 'Node297@4']

Entering Phase II
Entering Phase I
MMPC_beggining: 
369

Universe actual size:
9
CPC actual size:
1
CPC contents:
['Node298@4']

Universe actual size:
7
CPC actual size:
2
CPC contents:
['Node298@4', 'Node317@4']

Universe actual size:
3
CPC actual size:
3
CPC contents:
['Node298@4', 'Node317@4', 'Node304@3']

Universe actual size:
1
CPC actual size:
4
CPC contents:
['Node298@4', 'Node317@4', 'Node304@3', 'Node34@3']

Universe actual size:
0
CPC actual size:
5
CPC cont

Universe actual size:
10
CPC size before filtering: 1
CPC size after filtering: 1
['Node314@3']

Universe actual size:
6
CPC size before filtering: 3
Size set for community selection phase will be of maximum: 3
['Node314@3', 'Node325@3', 'Node297@4']
n_set of length: 3
CPC size after filtering: 2
['Node325@3', 'Node314@3']

Universe actual size:
5
CPC size before filtering: 3
Size set for community selection phase will be of maximum: 3
['Node314@3', 'Node325@3', 'Node298@4']
n_set of length: 3
CPC size after filtering: 2
['Node325@3', 'Node314@3']

Universe actual size:
4
CPC size before filtering: 3
Size set for community selection phase will be of maximum: 3
['Node314@3', 'Node325@3', 'Node300@4']
n_set of length: 3
CPC size after filtering: 2
['Node325@3', 'Node314@3']

Universe actual size:
3
CPC size before filtering: 3
Size set for community selection phase will be of maximum: 3
['Node314@3', 'Node325@3', 'Node308@3']
n_set of length: 3
CPC size after filtering: 3
['Node308@3', '

Universe actual size:
8
CPC size before filtering: 1
CPC size after filtering: 1
['Node299@3']

Universe actual size:
7
CPC size before filtering: 2
Size set for community selection phase will be of maximum: 2
['Node299@3', 'Node13@3']
n_set of length: 2
CPC size after filtering: 2
['Node13@3', 'Node299@3']

Universe actual size:
6
CPC size before filtering: 3
Size set for community selection phase will be of maximum: 3
['Node299@3', 'Node8@3', 'Node13@3']
n_set of length: 3
CPC size after filtering: 2
['Node8@3', 'Node299@3']

Universe actual size:
5
CPC size before filtering: 3
Size set for community selection phase will be of maximum: 3
['Node299@3', 'Node8@3', 'Node164@3']
n_set of length: 3
CPC size after filtering: 3
['Node164@3', 'Node8@3', 'Node299@3']

Universe actual size:
3
CPC size before filtering: 4
Size set for community selection phase will be of maximum: 3
['Node299@3', 'Node8@3', 'Node164@3', 'Node156@3']
n_set of length: 4
CPC size after filtering: 4
['Node156@3', 'N